# Práctica 4: Procesamieto de audio

### **Participantes:**
- Gerardo León Quintana
- Susana Suárez Mendoza

## Ejercicio 1:

Construir un identificador de notas musicales. Es decir; en su versión más sencilla (y suficiente) la entrada es un sonido con una sola nota musical y debe identificar cuál es. Por simplicidad elija un único instrumento para la identificación.

## Ejercicio 2:

Construir una pequeña aplicación que permita operar con diferentes filtros (con un selector) y trabajar con varios umbrales. (uno para los filtros pasa-bajo y pasa-alto y dos para los filtros pasa-banda y rechaza-banda). Demuestre su funcionalidad con señales ruidosas. Muestre en cada filtrado la señal original y filtrada en el dominio temporal y en el dominio de la frecuencia.

# gerardo branch

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, freqz
from scipy.io import wavfile
import tkinter as tk
from tkinter import filedialog, ttk
import vlc

In [3]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandstop(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandstop')
    return b, a


In [4]:
# Función para aplicar un filtro
def apply_filter(data, b, a):
    y = lfilter(b, a, data)
    return y

# Función para plotear en el dominio temporal y de frecuencia
def plot_signal(time, signal, fs, title):
    plt.figure(figsize=(12, 6))
    
    # Dominio temporal
    plt.subplot(2, 1, 1)
    plt.plot(time, signal)
    plt.title(f'{title} - Time Domain')
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    
    # Dominio de frecuencia
    plt.subplot(2, 1, 2)
    freqs = np.fft.fftfreq(len(signal), 1/fs)
    fft_spectrum = np.fft.fft(signal)
    plt.plot(freqs[:len(freqs)//2], np.abs(fft_spectrum)[:len(freqs)//2])
    plt.title(f'{title} - Frequency Domain')
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Amplitude')
    
    plt.tight_layout()
    plt.show()


In [ ]:

input_wav_file = 'crazy_frog_axel.wav'
fs, data = wavfile.read(input_wav_file)

# Normalizar si es necesario
if data.dtype == np.int16:
    data = data / 32768.0
elif data.dtype == np.int32:
    data = data / 2147483648.0

# Si el archivo de audio es estéreo, seleccionar un canal
if len(data.shape) == 2:
    data = data[:, 0]

# Crear vector de tiempo
t = np.arange(0, len(data) / fs, 1 / fs)

# Parámetros del filtro
order = 6

# Filtros y sus parámetros
filters = {
    'lowpass': {'func': butter_lowpass, 'params': {'cutoff': 1000}},
    'highpass': {'func': butter_highpass, 'params': {'cutoff': 1000}},
    'bandpass': {'func': butter_bandpass, 'params': {'lowcut': 500, 'highcut': 1500}},
    'bandstop': {'func': butter_bandstop, 'params': {'lowcut': 500, 'highcut': 1500}}
}

plot_signal(t, data, fs, 'Original Signal')

for filter in filters:
    b, a = filters[filter]['func'](fs=fs, order=order, **filters[filter]['params'])
    filtered_data = apply_filter(data, b, a)
    plot_signal(t, filtered_data, fs, f'Filtered Signal - {filter.capitalize()}')

    filtered_data_int = np.int16(filtered_data * 32767)
    output_wav_file = f'{filter}_filtered.wav'
    wavfile.write(output_wav_file, fs, filtered_data_int)


In [9]:
# Clase para la interfaz gráfica
class AudioPlayer:
    def __init__(self, master):
        self.master = master
        self.master.title("Reproductor de Audio")
        self.master.geometry("300x150")

        # Inicializar el reproductor VLC
        self.player = None
        self.is_playing = False

        # Botón para elegir archivo
        self.select_button = tk.Button(self.master, text="Seleccionar Archivo", command=self.select_file)
        self.select_button.pack(pady=20)

        # Botón de reproducir/pausar
        self.play_button = tk.Button(self.master, text="Reproducir", command=self.toggle_play, state=tk.DISABLED)
        self.play_button.pack(pady=20)

    def select_file(self):
        # Abrir un cuadro de diálogo para seleccionar un archivo de audio
        file_path = filedialog.askopenfilename(title="Seleccionar Archivo de Audio", filetypes=[("Audio Files", "*.mp3;*.wav;*.ogg;*.flac")])
        if file_path:
            if self.player is not None and self.is_playing:
                self.player.stop()  # Detener el audio actual si está en reproducción

            self.player = vlc.MediaPlayer(file_path)
            self.play_button.config(state=tk.NORMAL)  # Habilitar el botón de reproducir
            self.is_playing = False  # Reiniciar estado de reproducción
            self.play_button.config(text="Reproducir")

    def toggle_play(self):
        if self.player is None:
            return  # No hay archivo seleccionado

        if self.is_playing:
            self.player.pause()
            self.play_button.config(text="Continuar")
        else:
            if self.player.is_playing():  # Si ya hay un audio en reproducción, detenerlo
                self.player.stop()  # Detener el audio actual
            self.player.play()  # Reproducir el nuevo audio
            self.play_button.config(text="Pausar")
        self.is_playing = not self.is_playing

# Crear la ventana principal
if __name__ == "__main__":
    root = tk.Tk()
    app = AudioPlayer(root)
    root.mainloop()
